# Quantum Gram-Schmidt Process

# Lesson Plan Draft
## Linear Algebra Review
- What is complete basis?
- What is row basis?
- What is complete row basis?
- Recall: Relation between linear independence and complete basis

## Gram-Schmidt Process
- What is the Gram-Schmidt process?
- Application of Gram-Schmidt process to quantum states (in ML and other areas)

### Easy example: State discrimination
- State discrimination problem
- Application in query complexity
- Problem when applying Gram-Schimidt process to quantum states (exponential blowup)

## What is the quantum Gram-Schmidt process?
- High-level overview of the quantum Gram-Schmidt process
- The math
    - exercises (?)
- The circuit
    - exercises (?)
- Solved example: Quantum Gram-Schmidt process

### Application 1: Back to State discrimination

### Application 2: Quantum Gram-Schmidt process for quantum states

# What is Quantum Gram-Schmidt Process?
- Quantum Gram-Schmidt algorithm is closely related to Classical Gram-Schmidt process but does something entirely different. QGS is used to generate a **complete row basis** by performing a quantum version of *adapting sampling* on the rows.
    - rows which have a larger orthogonal part to the previously sampled row submatrix, will be sampled with a larger probability.
    - This ensures that the complete basis is non-singular.